In [ ]:
import Adafruit_DHT as dht
import time
import mysql.connector as mysql
import paho.mqtt.client as mqtt
import struct

sensor = dht.DHT22
db = mysql.connect(host="54.243.27.178", user="usuario", passwd="contraseña", database="BDDWINE")
cursor = db.cursor()
query = 'INSERT INTO ReadDHT22(Atemperatura, humedad) VALUE(%s, %s)'

cliente = mqtt.Client()
cliente.username_pw_set(username="tesis",password="contraseña")
cliente.connect("54.243.27.178",1883)

datos = [0.0,0.0]
try:
    while True:
        h, t = dht.read_retry(sensor, 4)
        if h is not None and t is not None:
            temperature = round(t, 2)
            humidity = round(h, 2)
            print(f"Temperatura = {temperature} oC, Humedad = {humidity} %")
            cursor.execute(query,(temperature, humidity))
            db.commit()
            print("Lectura insertada en la base de datos")
            # Preparar los datos como bytes
            datos[0] = temperature
            datos[1] = humidity
            # Publicar los datos en el broker MQTT
            cliente.publish("demo", struct.pack("ff",*datos))
        else:
            print("Error al leer el sensor")
        time.sleep(14400)
        
except KeyboardInterrupt:
    print("Fin de programa")

finally:
    if db is not None and db.is_connected():
        cursor.close()
        print("Conexion a la base de datos cerrada")
